# Les Versions de l'article 9

In [1]:
#import des modules

#lecture des json
import json 

#affichage des valeurs identées
import pprint
pp = pprint.PrettyPrinter(indent=4, )

#affichage des graphs
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:

#chargement de l'article 9
with open("article9.json", "r") as f:
    article9 = json.load(f)

* On va stocker le détail des participations de l'article 9 dans version_originale de la manière suivante:

In [4]:
version_originale = {
                "date": article9["created_at"],
                "link": article9["article_link"],
                "slug": article9["article_link"].split("/")[-1],
                "title": article9["article_link"].split("/")[-1].replace("-", " "),
                "text": article9['body'],
                "author": article9["author"],
                "votes":[], 
                "arguments":article9["arguments"], 
                "votes_arguments": [],
                "sources":article9["sources"],
                "votes_sources": [],
               }

* On va stocker les versions complémentaires dans versions

In [5]:
versions = article9["versions"]


et reformater les versions de la même manière que dans la version originale


In [6]:
versions_c = []
for v in versions:
    vn = {"date":v["created_at"],
"link":v["link"],
"slug":v["slug"],
"title":v["title"],
"text": v['comment'],
"author": v["author"],
          "votes":[], 
"arguments":[], 
"votes_arguments": [],
"sources":[],
"votes_sources": []}
    versions_c.append(vn)

* On ajoute ensuite la version originale

In [7]:
versions_c.append(version_originale)

* Maintenant on va le transformer en dictionnaire avec pour clé le slug

In [8]:
versions_d = {}
for v in versions_c:
    versions_d[v["slug"]] = v
    


In [9]:
print len(versions_d)

109


# Les electeurs de l'article 9

On charge le fichiers participants qui contient toutes les actions de chaque participant

In [10]:

#Ici on ne s'intéresse qu'aux votes
#qui sont détaillés par participants
with open("participants.json", "r") as f:
    data = json.load(f)
    participants = data["participants"]
#Verifions le nombre de participants

print len(participants), "participants sur l'ensemble des articles"
#les noms des participants
electeurs = [n.keys()[0] for n in participants]


21329 participants sur l'ensemble des articles


On charge ensuite seulement les votes des électeurs de l'article 9 dans votes

In [11]:
art = "article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics"
votes = []
for user in participants:
    #accesible via son nom
    name =  user.keys()[0]
    #récupérer les votes de l'utilisateur dans "votes"
    #filtrer ceux qui ont des votes qui correspondent à l'article
    for vote in user[name]["votes"]:
        if art in vote["link"]:
            vote["electeur"] = name
            votes.append(vote)
            

In [12]:
print len(votes), "votes en tout genre"

15547 votes en tout genre


In [13]:
for vote in votes:
    url = vote["link"]
    slug = url.split("/")[-1]
    #vote sur un argument        
    if "#arg" in slug:
        vote["slug"], vote["id"] = slug.split("#")
        versions_d[vote["slug"]]["votes_arguments"].append(vote)
        
        #vote sur une source
    elif "#source" in slug:
        vote["slug"], vote["id"] = slug.split("#")
        versions_d[vote["slug"]]["votes_sources"].append(vote)
        
    #vote sur un article
    else:
        vote["slug"] = slug
        versions_d[vote["slug"]]["votes"].append(vote)
        

In [14]:
print len(versions_d[art]["votes"])

3334


In [15]:
print len(versions_d[art]["sources"])

22


In [16]:
print len(versions_d[art]["arguments"])

132


In [17]:
print len(versions_d[art]["votes_sources"])

130


In [18]:
print len(versions_d[art]["votes_arguments"])

1155


In [22]:
auteur_electeur = []

for version, data in versions_d.items():
    for vote in data["votes"]:
        auteur_electeur.append([version, data["author"], vote["electeur"], vote["opinion"]])

In [23]:
print len(auteur_electeur)

12851


In [25]:
with open("votes_auteur_electeur.csv", "w") as f:
    f.write("version\tauteur\telecteur\tvote\n")
    for line in auteur_electeur:
        f.write(("\t").join(line)+"\n")
        

Reseau d'affinité electeur auteur
Deux acteurs sont reliés dès lors que l'un d'entre eux a voté **pour** une version proposé par l'autre.

Matrice de coocurrence: on multiplie la matrice par sa transposition
de sorte que f(x) = x(f)
soit a->b et b->a ou encore 
paul a voté pour pierre & pierre a voté pour paul
on doit garantir que dans notre résultat les deux formes ne cooexistent pas
soit la diagonale doit etre enlevé ou encore garantir qu'une seule des options existe



In [41]:
auteur_vs_electeur = { "auteur":[], "electeur":[], "votes":[], "versions":[]}
for line in auteur_electeur:
    auteur_vs_electeur["version"].append(line[0])
    auteur_vs_electeur["auteur"].append(line[1])
    auteur_vs_electeur["electeur"].append(line[2])
    auteur_vs_electeur["vote"].append(line[3])
    

In [46]:
auteur_vs_electeur
    

{'version': [u'attention-a-la-qualite-et-a-la-securite-des-textes-scientifiques-qui-doivent-etre-valides-en-amont-promus-et-enrichis-ensuite-par-le-biais-de-bases-de-donnees', u'attention-a-la-qualite-et-a-la-securite-des-textes-scientifiques-qui-doivent-etre-valides-en-amont-promus-et-enrichis-ensuite-par-le-biais-de-bases-de-donnees', u'attention-a-la-qualite-et-a-la-securite-des-textes-scientifiques-qui-doivent-etre-valides-en-amont-promus-et-enrichis-ensuite-par-le-biais-de-bases-de-donnees', u'attention-a-la-qualite-et-a-la-securite-des-textes-scientifiques-qui-doivent-etre-valides-en-amont-promus-et-enrichis-ensuite-par-le-biais-de-bases-de-donnees', u'attention-a-la-qualite-et-a-la-securite-des-textes-scientifiques-qui-doivent-etre-valides-en-amont-promus-et-enrichis-ensuite-par-le-biais-de-bases-de-donnees', u'attention-a-la-qualite-et-a-la-securite-des-textes-scientifiques-qui-doivent-etre-valides-en-amont-promus-et-enrichis-ensuite-par-le-biais-de-bases-de-donnees', u'attenti

In [42]:
import pandas as pd

In [45]:
df = pd.DataFrame(auteur_vs_electeur)
#print df

df_asint = df.astype(int)
coocc = df_asint.T.dot(df_asint)
print coocc

ValueError: invalid literal for long() with base 10: 'dominiqueillien'